In [902]:
import pandas as pd
import numpy as np
import os
import math
import json

In [903]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
from pandas.errors import SettingWithCopyWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))
warnings.simplefilter(action='ignore', category=(FutureWarning))

In [904]:
defense = pd.read_csv("../match_logs/Big5@22-23@defense.csv")
gca = pd.read_csv("../match_logs/Big5@22-23@gca.csv")
keeper = pd.read_csv("../match_logs/Big5@22-23@keeper.csv")
misc = pd.read_csv("../match_logs/Big5@22-23@misc.csv")
passing = pd.read_csv("../match_logs/Big5@22-23@passing.csv")
passing_types = pd.read_csv("../match_logs/Big5@22-23@passing_types.csv")
possession = pd.read_csv("../match_logs/Big5@22-23@possession.csv")
shooting = pd.read_csv("../match_logs/Big5@22-23@shooting.csv")

In [905]:
team_ratings = pd.read_csv("../players_db/fm23/team_ratings.csv")

In [906]:
team_ratings_cols = team_ratings.columns

In [937]:
team_ratings_cols

Index(['Club', 'GK', 'Based', 'Division', 'Club_id', 'League_id', 'tpr',
       'Aggression', 'Anticipation', 'Bravery', 'Composure', 'Concentration',
       'Decisions', 'Determination', 'Flair', 'Leadership', 'Off the Ball',
       'Positioning', 'Teamwork', 'Vision', 'Work Rate', 'Acceleration',
       'Agility', 'Balance', 'Jumping Reach', 'Fitness', 'Pace', 'Stamina',
       'Strength', 'Corners', 'Crossing', 'Dribbling', 'Finishing',
       'First Touch', 'Free Kick Taking', 'Heading', 'Long Shots',
       'Long Throws', 'Marking', 'Passing', 'Penalty Taking', 'Tackling',
       'Technique', 'fbref_name'],
      dtype='object')

In [907]:
tpr_cols = []
except_cols = ["Club","Based","Division","Club_id","League_id","fbref_name"]
for col in team_ratings_cols:
    if col not in except_cols:
        for xy in ["x","y"]:
            xy_col = f"{col}_{xy}"
            tpr_cols.append(xy_col)

<br><br><br><br><br><br><br><br>
<h1 style="color:orange;background:blue;">@.  Logs to Regression</h1>

In [908]:
CURRENT_DATAFRAME_NAME = "shooting"

In [909]:
CURRENT_DATAFRAME = locals()[CURRENT_DATAFRAME_NAME]
CURRENT_DATAFRAME["Unnamed: 0"] = "itsanobject"

In [910]:
# CURRENT_DATAFRAME["Unique_Matchday_id"] = ((CURRENT_DATAFRAME.Club).apply(hash)+(CURRENT_DATAFRAME.Opponent).apply(hash)).apply(str)
# unique_indexes = CURRENT_DATAFRAME["Unique_Matchday_id"].drop_duplicates().index
# CURRENT_DATAFRAME = CURRENT_DATAFRAME.loc[unique_indexes]

In [911]:
DATAFRAME_COLUMNS = list(CURRENT_DATAFRAME.columns)
NUMERIC_DATAFRAME_COLUMNS = list(CURRENT_DATAFRAME.select_dtypes(exclude="object").columns)

In [912]:
CURRENT_DATAFRAME[NUMERIC_DATAFRAME_COLUMNS].describe().loc[['count','min', 'mean', 'max']].T[["mean"]].apply(round)

mean
GF        1.0
GA        1.0
Gls       1.0
Sh       12.0
SoT       4.0
SoT%     34.0
G/Sh      0.0
G/SoT     0.0
Dist     18.0
FK        0.0
PK        0.0
PKatt     0.0
xG        1.0
npxG      1.0
npxG/Sh   0.0
G-xG     -0.0
np:G-xG  -0.0

In [913]:
merged_df_1 = pd.merge(CURRENT_DATAFRAME, team_ratings, on="Club", how="inner")  
merged_df = pd.merge(merged_df_1, team_ratings, left_on="Opponent", right_on="fbref_name", how="inner")  

In [914]:
merged_df["Versus_Title"] = merged_df.Club_x + " plays vs " + \
                             merged_df.Club_y + " as " + merged_df.Venue + " team"

merged_df["tpr_diff_abs"] = abs(merged_df.tpr_x - merged_df.tpr_y)
merged_df["tpr_diff"] = abs(merged_df.tpr_x - merged_df.tpr_y)

In [915]:
# merged_df = merged_df.query("tpr_diff < 3")
merged_df = merged_df.query("tpr_diff > 2")
merged_df.describe()

GF           GA          Gls           Sh          SoT  \
count  2782.000000  2782.000000  2782.000000  2782.000000  2782.000000   
mean      1.397196     1.397196     1.352983    12.472682     4.204889   
std       1.273481     1.273481     1.252595     5.215921     2.444730   
min       0.000000     0.000000     0.000000     1.000000     0.000000   
25%       0.000000     0.000000     0.000000     9.000000     2.000000   
50%       1.000000     1.000000     1.000000    12.000000     4.000000   
75%       2.000000     2.000000     2.000000    16.000000     6.000000   
max       9.000000     9.000000     8.000000    35.000000    20.000000   

              SoT%         G/Sh        G/SoT         Dist           FK  ...  \
count  2782.000000  2782.000000  2702.000000  2782.000000  2782.000000  ...   
mean     34.070417     0.104849     0.296902    17.609526     0.416966  ...   
std      15.923265     0.105429     0.269648     3.135258     0.643145  ...   
min       0.000000     0.000000     0.000000     7.500000     0.000000  ...   
25%      23.100000     0.000000     0.000000    15.500000     0.000000  ...   
50%      33.300000     0.080000     0.250000    17.500000     0.000000  ...   
75%      43.800000     0.170000     0.500000    19.300000     1.000000  ...   
max     100.000000     0.800000     1.000000    54.900000     4.000000  ...   

         Heading_y  Long Shots_y  Long Throws_y    Marking_y    Passing_y  \
count  2867.000000   2867.000000    2867.000000  2867.000000  2867.000000   
mean     70.911406     70.017789      74.376352    75.719916    69.107429   
std       7.240401      7.979039       7.957470     7.683055     7.685662   
min      50.000000     50.000000      50.000000    50.000000    50.000000   
25%      65.000000     64.000000      68.000000    69.000000    64.000000   
50%      71.000000     70.000000      74.000000    74.000000    67.000000   
75%      74.000000     75.000000      81.000000    80.000000    74.000000   
max      92.000000     92.000000      92.000000    92.000000    92.000000   

       Penalty Taking_y   Tackling_y  Technique_y  tpr_diff_abs     tpr_diff  
count       2867.000000  2867.000000  2867.000000   2867.000000  2867.000000  
mean          71.024765    73.785839    70.405651      8.875828     8.875828  
std            7.436712     8.771276     8.357738      5.165840     5.165840  
min           50.000000    50.000000    50.000000      3.000000     3.000000  
25%           66.000000    68.000000    63.000000      5.000000     5.000000  
50%           71.000000    74.000000    70.000000      7.000000     7.000000  
75%           75.000000    80.000000    76.000000     12.000000    12.000000  
max           92.000000    92.000000    92.000000     30.000000    30.000000  

[8 rows x 99 columns]

In [916]:
MERGED_NUMERIC_DATAFRAME_COLUMNS = list(merged_df.select_dtypes(exclude="object").columns)

In [917]:
reg_df = merged_df[['Club_x','Club_y',"Versus_Title","Date"] + MERGED_NUMERIC_DATAFRAME_COLUMNS] 
reg_df.iloc[0]

Club_x                                                Manchester City
Club_y                                                West Ham United
Versus_Title        Manchester City plays vs West Ham United as Aw...
Date                                                       2022-08-07
GF                                                                2.0
                                          ...                        
Penalty Taking_y                                                   68
Tackling_y                                                         84
Technique_y                                                        75
tpr_diff_abs                                                       13
tpr_diff                                                           13
Name: 0, Length: 103, dtype: object

<br><br><br><br><br><br><br><br>
<h2 style="color:white;background:red;">  Regression to get Feature Importance</h2>

In [918]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

In [919]:
formula_df = pd.DataFrame(index=tpr_cols)
np.random.seed(0)
df = reg_df.dropna(axis=0)
NUMERIC_DATAFRAME_COLUMNS = ["Sh"]

for attribute in NUMERIC_DATAFRAME_COLUMNS:
    X = df[tpr_cols]
    Y = df[[attribute]]
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
    model = LinearRegression()
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    
    coef_column_name = f"{attribute}_coef"
    intercept_column_name = f"{attribute}_intercept"
    formula_df[coef_column_name] = model.coef_.flatten()
    formula_df[intercept_column_name] = model.intercept_[0]

    mse = mean_squared_error(Y_test, Y_pred)
    r2 = r2_score(Y_test, Y_pred)
    print(f'{attribute} --- Mean Squared Error: {mse}')
    print(f'{attribute} --- R^2 Score: {r2}')
    # print("\n")

Sh --- Mean Squared Error: 21.04358902874593
Sh --- R^2 Score: 0.22332780241536732


In [920]:
formula_df["Sh_abs_coef"] = formula_df["Sh_coef"].apply(abs)
formula_df.query("Sh_abs_coef > 0.05").sort_values(by="Sh_coef",ascending=False)

Sh_coef  Sh_intercept  Sh_abs_coef
tpr_x               0.148712      11.64086     0.148712
Acceleration_x      0.121252      11.64086     0.121252
GK_x                0.120032      11.64086     0.120032
Passing_y           0.117235      11.64086     0.117235
Vision_y            0.109632      11.64086     0.109632
Strength_y          0.106068      11.64086     0.106068
Finishing_x         0.105221      11.64086     0.105221
Technique_x         0.101112      11.64086     0.101112
Tackling_y          0.080203      11.64086     0.080203
Composure_x         0.078735      11.64086     0.078735
Corners_x           0.077421      11.64086     0.077421
Aggression_y        0.073096      11.64086     0.073096
Stamina_y           0.069129      11.64086     0.069129
Anticipation_y      0.064402      11.64086     0.064402
Composure_y         0.062730      11.64086     0.062730
Flair_y             0.060489      11.64086     0.060489
Work Rate_x         0.057668      11.64086     0.057668
Off the Ball_x      0.050193      11.64086     0.050193
Passing_x          -0.059054      11.64086     0.059054
Anticipation_x     -0.059078      11.64086     0.059078
Crossing_y         -0.064614      11.64086     0.064614
Decisions_x        -0.067536      11.64086     0.067536
Marking_y          -0.068461      11.64086     0.068461
Off the Ball_y     -0.074749      11.64086     0.074749
Free Kick Taking_x -0.075026      11.64086     0.075026
Agility_y          -0.081575      11.64086     0.081575
Work Rate_y        -0.082069      11.64086     0.082069
Technique_y        -0.101360      11.64086     0.101360
Stamina_x          -0.110784      11.64086     0.110784
tpr_y              -0.391377      11.64086     0.391377

In [921]:
# with pd.ExcelWriter(f"STANDARD_SCALER---------------{CURRENT_DATAFRAME_NAME}_Standardized_coefficients.xlsx") as writer:
#     formula_df[formula_df.index.str.contains("_x")].to_excel(writer, index=True, sheet_name="x")
#     formula_df[formula_df.index.str.contains("_y")].to_excel(writer, index=True, sheet_name="y")

<br><br><br><br><br><br><br><br>
<h2 style="color:white;background:green;">  RESEARCHING MATHEMATICS</h2>

In [922]:
#

<br><br><br><br>

<br><br><br><br><br><br><br><br>
<h2 style="color:#33FF49;background:blue;">  Regression for One Atrribute</h2>

In [923]:
attribute = "Sh"

In [924]:
tpr_x_dict = {
    "tpr_x": None,
    "Acceleration_x": None,
    "Technique_x": None,
    "Finishing_x": None,
    "Composure_x": None,
    "Agility_x": None,
    "Long Shots_x": None,
    "Off the Ball_x": None,
    "Teamwork_x": None,
    "Work Rate_x": None,
    "First Touch_x": None
}

In [925]:
tpr_y_dict = {
    "tpr_y": None,
    "Acceleration_y": None,
    "Off the Ball_y": None,
    "Agility_y": None,
    "Work Rate_y": None,
    "Decisions_y": None,
    "Teamwork_y": None,
    "Balance_y": None,
    "Positioning_y": None,
    "Marking_y": None
}

In [926]:
# https://fbref.com/en/matches/5965a1a5/Bayern-Munich-Bochum-February-11-2023-Bundesliga
# row_index= 3221 # for Bayern
# row_index2222= 3758 # for Bochum

row_index = 0 # 687 # for Liverpool
row_index2222 = 356 # for Arsenal

x_cols = tpr_x_dict.keys()
y_cols = tpr_y_dict.keys()

tpr_x_custom_data = reg_df[x_cols].iloc[row_index].to_dict()
tpr_y_custom_data = reg_df[y_cols].iloc[row_index].to_dict()

custom_data = {**tpr_x_custom_data, **tpr_y_custom_data}
custom_cols = custom_data.keys()

In [927]:
df.iloc[687]

Club_x                                              Udinese
Club_y                                            Cremonese
Versus_Title        Udinese plays vs Cremonese as Away team
Date                                             2022-10-30
GF                                                      0.0
                                     ...                   
Penalty Taking_y                                         64
Tackling_y                                               76
Technique_y                                              61
tpr_diff_abs                                              5
tpr_diff                                                  5
Name: 891, Length: 103, dtype: object

In [928]:
# q1 = reg_df.query("Versus_Title == 'Manchester City plays vs Liverpool as Home team'")
# q2 = reg_df.query("Versus_Title == 'Liverpool plays vs Manchester City as Home team'")
# queries = ["", q1, q2]
# query_no = 2
# print(queries[query_no].iloc[0].Date)
# queries[query_no].iloc[0].name
# queries[query_no].iloc[0].name

In [929]:
special_formula_df = pd.DataFrame(index=custom_cols)

df_to_custom = df.copy()
X = df_to_custom[custom_cols]
Y = df_to_custom[[attribute]]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
model = LinearRegression()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

coef_column_name = f"{attribute}_coef"
intercept_column_name = f"{attribute}_intercept"
special_formula_df[coef_column_name] = model.coef_.flatten()
special_formula_df[intercept_column_name] = model.intercept_[0]

mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

In [930]:
special_formula_df

Sh_coef  Sh_intercept
tpr_x           0.027249       8.08039
Acceleration_x  0.086299       8.08039
Technique_x     0.070744       8.08039
Finishing_x     0.042388       8.08039
Composure_x     0.022539       8.08039
Agility_x      -0.001659       8.08039
Long Shots_x    0.008991       8.08039
Off the Ball_x  0.027584       8.08039
Teamwork_x      0.006078       8.08039
Work Rate_x     0.011567       8.08039
First Touch_x  -0.073716       8.08039
tpr_y          -0.192540       8.08039
Acceleration_y  0.017872       8.08039
Off the Ball_y  0.004663       8.08039
Agility_y      -0.050023       8.08039
Work Rate_y    -0.000661       8.08039
Decisions_y    -0.019030       8.08039
Teamwork_y     -0.012435       8.08039
Balance_y       0.056552       8.08039
Positioning_y   0.039908       8.08039
Marking_y      -0.011505       8.08039

In [931]:
custom_tpr_df = pd.DataFrame(custom_data, index=["tpr_value"]).T

In [932]:
custom_tpr_df[f"{attribute}_coef"] = special_formula_df[f"{attribute}_coef"][custom_cols].tolist()
custom_tpr_df[f"{attribute}_intercept"] = special_formula_df[f"{attribute}_intercept"][custom_cols].tolist()
custom_tpr_df["coef_result"] = custom_tpr_df[f"{attribute}_coef"] * custom_tpr_df["tpr_value"]
custom_tpr_df["formula_result"] = custom_tpr_df["coef_result"].sum() + custom_tpr_df[f"{attribute}_intercept"].iloc[0]
custom_tpr_df["formula_result"] = round(custom_tpr_df["formula_result"].iloc[0])

In [933]:
custom_tpr_df

tpr_value   Sh_coef  Sh_intercept  coef_result  formula_result
tpr_x                  92  0.027249       8.08039     2.506868              15
Acceleration_x         85  0.086299       8.08039     7.335411              15
Technique_x            86  0.070744       8.08039     6.083962              15
Finishing_x            90  0.042388       8.08039     3.814933              15
Composure_x            88  0.022539       8.08039     1.983395              15
Agility_x              89 -0.001659       8.08039    -0.147686              15
Long Shots_x           79  0.008991       8.08039     0.710251              15
Off the Ball_x         82  0.027584       8.08039     2.261924              15
Teamwork_x             72  0.006078       8.08039     0.437613              15
Work Rate_x            85  0.011567       8.08039     0.983220              15
First Touch_x          85 -0.073716       8.08039    -6.265876              15
tpr_y                  79 -0.192540       8.08039   -15.210685              15
Acceleration_y         75  0.017872       8.08039     1.340382              15
Off the Ball_y         74  0.004663       8.08039     0.345072              15
Agility_y              71 -0.050023       8.08039    -3.551642              15
Work Rate_y            78 -0.000661       8.08039    -0.051579              15
Decisions_y            72 -0.019030       8.08039    -1.370152              15
Teamwork_y             72 -0.012435       8.08039    -0.895339              15
Balance_y              82  0.056552       8.08039     4.637227              15
Positioning_y          84  0.039908       8.08039     3.352308              15
Marking_y              80 -0.011505       8.08039    -0.920413              15

In [934]:
custom_tpr_df.loc[:, "tpr_value"]

tpr_x             92
Acceleration_x    85
Technique_x       86
Finishing_x       90
Composure_x       88
Agility_x         89
Long Shots_x      79
Off the Ball_x    82
Teamwork_x        72
Work Rate_x       85
First Touch_x     85
tpr_y             79
Acceleration_y    75
Off the Ball_y    74
Agility_y         71
Work Rate_y       78
Decisions_y       72
Teamwork_y        72
Balance_y         82
Positioning_y     84
Marking_y         80
Name: tpr_value, dtype: int64

In [935]:
custom_tpr_df.index

Index(['tpr_x', 'Acceleration_x', 'Technique_x', 'Finishing_x', 'Composure_x',
       'Agility_x', 'Long Shots_x', 'Off the Ball_x', 'Teamwork_x',
       'Work Rate_x', 'First Touch_x', 'tpr_y', 'Acceleration_y',
       'Off the Ball_y', 'Agility_y', 'Work Rate_y', 'Decisions_y',
       'Teamwork_y', 'Balance_y', 'Positioning_y', 'Marking_y'],
      dtype='object')

In [936]:
json.loads('["x","y","z"]')


['x', 'y', 'z']

<br><br><br><br><br><br><br><br>
<h2 style="color:blue;background:aqua;">  Error Rates</h2>